In [1]:
d={"key":"value" , "key2":"value2"}


In [2]:
from box import ConfigBox


In [3]:
d1=ConfigBox(d)
d1.key


'value'

In [ ]:
import os
from pathlib import Path
from dataclasses import dataclass
import pandas as pd
from tqdm import tqdm
import torch
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from datasets import Dataset
from evaluate import load
import yaml
from text_summarizer.logging import logger

# =======================
# CONFIG DATA CLASS
# =======================
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    tokenizer_path: Path
    data_path: Path
    metric_file_name: Path
    column_text: str = "article"
    column_summary: str = "highlights"

# =======================
# CONFIGURATION MANAGER
# =======================
class ConfigurationManager:
    def __init__(self, config_path: Path):
        if not config_path.exists():
            raise FileNotFoundError(f"Config file not found: {config_path}")
        with open(config_path, "r") as f:
            self.config = yaml.safe_load(f)

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        cfg = self.config["model_evaluation"]
        return ModelEvaluationConfig(
            root_dir=Path(cfg["root_dir"]),
            model_path=Path(cfg["model_path"]),
            tokenizer_path=Path(cfg["tokenizer_path"]),
            data_path=Path(cfg["data_path"]),
            metric_file_name=Path(cfg["metric_file_name"]),
            column_text=cfg.get("column_text", "article"),
            column_summary=cfg.get("column_summary", "highlights")
        )

# =======================
# MODEL EVALUATION CLASS
# =======================
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load model safely on Windows
        model_dir = str(self.config.model_path.as_posix())
        tokenizer_dir = str(self.config.tokenizer_path.as_posix())

        logger.info(f"Loading model from {model_dir}")
        self.model = T5ForConditionalGeneration.from_pretrained(model_dir).to(self.device)

        logger.info(f"Loading tokenizer from {tokenizer_dir}")
        self.tokenizer = T5TokenizerFast.from_pretrained(tokenizer_dir)

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i + batch_size]

    def calculate_metrics_on_test_ds(
        self, dataset, metric, batch_size=4
    ):
        self.model.eval()
        preds, refs = [], []

        for batch in tqdm(self.generate_batch_sized_chunks(list(range(len(dataset))), batch_size)):
            texts = [dataset[i][self.config.column_text] for i in batch]
            summaries = [dataset[i][self.config.column_summary] for i in batch]

            inputs = self.tokenizer(
                texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).to(self.device)

            with torch.no_grad():
                summary_ids = self.model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    max_length=150,
                    num_beams=2,
                    early_stopping=True
                )

            decoded_preds = self.tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
            preds.extend(decoded_preds)
            refs.extend(summaries)

        return metric.compute(predictions=preds, references=refs)

    def evaluate(self):
        logger.info("Loading ROUGE metric")
        metric = load("rouge")

        # Load test CSV directly
        test_csv_path = self.config.data_path
        if not test_csv_path.exists():
            raise FileNotFoundError(f"Test CSV not found: {test_csv_path}")
        logger.info(f"Loading test dataset from {test_csv_path}")
        df = pd.read_csv(test_csv_path)
        test_dataset = Dataset.from_pandas(df)

        logger.info("Calculating metrics on test dataset")
        results = self.calculate_metrics_on_test_ds(test_dataset, metric)
         

# inside evaluate() before saving
        self.config.metric_file_name.parent.mkdir(parents=True, exist_ok=True)

        pd.DataFrame([results]).to_csv(self.config.metric_file_name, index=False)
        logger.info(f"Saving metrics to {self.config.metric_file_name}")
        pd.DataFrame([results]).to_csv(self.config.metric_file_name, index=False)
        logger.info(f"Metrics saved to {self.config.metric_file_name}")
        logger.info("Evaluation complete!")

# =======================
# USAGE
# =======================
if __name__ == "__main__":
    os.chdir(r"C:\Users\Admin\Desktop\text-summarizer\text-summarizer-project")  # Windows-safe
    try:
        config_manager = ConfigurationManager(Path("config/config.yaml"))
        eval_config = config_manager.get_model_evaluation_config()
        evaluator = ModelEvaluation(eval_config)
        evaluator.evaluate()
    except Exception as e:
        raise e


2025-12-03 20:41:00,750 - INFO - 4053448843 - Loading model from artifacts/model_trainer/t5-summarizer
2025-12-03 20:41:01,237 - INFO - 4053448843 - Loading tokenizer from artifacts/model_trainer/tokenizer
2025-12-03 20:41:01,506 - INFO - 4053448843 - Loading ROUGE metric
2025-12-03 20:41:13,445 - INFO - 4053448843 - Loading test dataset from artifacts\data_ingestion\samsum-test.csv
2025-12-03 20:41:13,821 - INFO - 4053448843 - Calculating metrics on test dataset


205it [12:23,  3.62s/it]


2025-12-03 20:53:37,232 - INFO - rouge_scorer - Using default tokenizer.
2025-12-03 20:53:40,437 - INFO - 4053448843 - Saving metrics to artifacts\model_evaluation\metric.csv
2025-12-03 20:53:40,453 - INFO - 4053448843 - Metrics saved to artifacts\model_evaluation\metric.csv
2025-12-03 20:53:40,460 - INFO - 4053448843 - Evaluation complete!
